In [3]:
import baostock as bs
import pandas as pd
import os
from datetime import datetime, timedelta
import akshare as ak

# 初始化baostock
bs.login()

# 定义时间范围
start_date = '2008-01-01'
end_date = '2024-10-31'

# 获取每个月的第一个交易日和最后一个交易日
def get_monthly_trading_days(start_date, end_date):
    df = ak.stock_zh_a_daily(symbol="sz000001", start_date=start_date, end_date=end_date, adjust="qfq")
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    monthly_trading_days = {}
    for year_month, group in df.groupby([df.index.year, df.index.month]):
        first_trading_day = group.index.min().strftime('%Y-%m-%d')
        last_trading_day = group.index.max().strftime('%Y-%m-%d')
        monthly_trading_days[first_trading_day] = (first_trading_day, last_trading_day)
    
    return monthly_trading_days

# 获取每个月初的沪深300成分股列表
def get_monthly_hs300_stocks(monthly_trading_days):
    monthly_hs300_stocks = {}
    
    for first_trading_day in monthly_trading_days.keys():
        if first_trading_day not in monthly_hs300_stocks:
            rs = bs.query_hs300_stocks(first_trading_day)
            hs300_stocks = []
            while (rs.error_code == '0') & rs.next():
                hs300_stocks.append(rs.get_row_data()[1])
            monthly_hs300_stocks[first_trading_day] = hs300_stocks
    
    return monthly_hs300_stocks

# 保存成分股数据到本地
def save_hs300_stocks_to_local(monthly_hs300_stocks, directory='hs300_stocks'):
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    for date, stocks in monthly_hs300_stocks.items():
        file_path = os.path.join(directory, f'hs300_stocks_{date}.csv')
        df = pd.DataFrame(stocks, columns=['stock_code'])
        df.to_csv(file_path, index=False)

# 获取每个月的第一个交易日和最后一个交易日
monthly_trading_days = get_monthly_trading_days(start_date, end_date)

# 获取每个月初的沪深300成分股列表
monthly_hs300_stocks = get_monthly_hs300_stocks(monthly_trading_days)

# 保存成分股数据到本地
save_hs300_stocks_to_local(monthly_hs300_stocks)

# 退出baostock
bs.logout()

login success!
logout success!
